# FOREX Prediction

In [ ]:
import requests
import json

import numpy as np
import pandas as pd
import tensorflow as tf

## Data Processing

### Raw Data Collection Using API

In [ ]:
api_key = 'YknABlU4L9jZ3e7azGRzR71EHuvjxjuT' + 'null'

date_of_collection = '2022-11-04'
base_currency = 'EUR'

data = []

for year in range(1999, 2023):
  s_date = f'{year}-01-01'
  e_date = f'{year}-12-31'

  response = requests.get(f'https://api.apilayer.com/exchangerates_data/timeseries?start_date={s_date}&end_date={e_date}', headers={ 'redirect': 'follow', 'apiKey':  api_key})
  data.append(response.json())

In [ ]:
with open('raw_data2.json', 'w') as f:
  json.dump(data, f)

In [ ]:
api_key = 'YknABlU4L9jZ3e7azGRzR71EHuvjxjuT' + 'null'

response = requests.get(f'https://api.apilayer.com/exchangerates_data/symbols', headers={ 'redirect': 'follow', 'apiKey':  api_key})

symbols_data = response.json()

In [ ]:
with open('raw_symbols2.json', 'w') as f:
  json.dump(symbols_data, f)

### Converting to Pandas Dataframe

In [ ]:
date_of_collection = '2022-11-04'

with open('raw_data.json', 'r') as f1:
  data = json.load(f1)
with open('raw_symbols.json', 'r') as f2:
  symbols_data = json.load(f2)

In [ ]:
all_rates = dict()
for d in data:
  all_rates.update(d['rates'])

dates = [d for d in list(all_rates.keys()) if d < date_of_collection]

In [ ]:
all_symbols = symbols_data['symbols']
symbols = list(all_symbols.keys())

In [ ]:
dataset = []

for date in dates:
  row = [date]
  for symbol in symbols:
    row.append(all_rates[date].get(symbol, None))
  dataset.append(row)


In [ ]:
base_columns = ['date', *symbols]
base_df = pd.DataFrame(np.asarray(dataset), columns=base_columns)
base_df

In [ ]:
columns = ['date'] + [f'{s1}-{s2}' for s1 in symbols for s2 in symbols]
df = base_df.drop(columns=symbols)

for s1 in symbols:
  for s2 in symbols:
    df[f'{s1}-{s2}'] = base_df[s1] / base_df[s2]
    print(s1, s2)

In [ ]:
df

In [ ]:
df.to_pickle('data.pkl')

## Machine Learning With GRU

### Data Preparation